In [ ]:
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import xesmf as xe

In [ ]:
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%matplotlib inline
%load_ext autoreload
%autoreload 2

# xESMF tutorial non-linear grid notebook
https://github.com/JiaweiZhuang/xESMF/blob/master/doc/Curvilinear_grid.ipynb

In [ ]:
ds = xr.tutorial.open_dataset('rasm')#.load()
ds

In [ ]:
dr = ds['Tair']
dr

In [ ]:
plt.figure(figsize=(12,2));
ax = plt.axes(projection=ccrs.PlateCarree());
dr[0].plot.pcolormesh(ax=ax, x='xc', y='yc');
ax.coastlines();

In [ ]:
plt.scatter(ds['xc'], ds['yc'], s=0.01)  # plot grid locations
plt.ylim([-90, 90])
plt.xlabel('lon')
plt.ylabel('lat')

In [ ]:
ds = ds.rename({'xc': 'lon', 'yc': 'lat'})
ds

In [ ]:
ds_out = xe.util.grid_global(5, 4)
ds_out  # contains lat/lon values of cell centers and boundaries.

In [ ]:
regridder = xe.Regridder(ds, ds_out, 'bilinear')
dr_out = regridder(dr)

In [ ]:
dr_out

In [ ]:
plt.figure(figsize=(12,4));
ax = plt.axes(projection=ccrs.PlateCarree())
dr_out[0].plot.pcolormesh(ax=ax, x='lon', y='lat');
ax.coastlines();

In [ ]:
regridder.clean_weight_file()  # clean-up

## CESM grids: `ocn` to `ocn_rect`

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from grid import generate_lats_lons
from paths import file_ex_ocn_rect, file_HadISST, file_RMASK_ocn, file_RMASK_ocn_rect, file_RMASK_ocn_had, file_ex_ocn_lpd

In [ ]:
generate_lats_lons()

In [ ]:
RMASK_ocn = xr.open_dataarray(file_RMASK_ocn, decode_times=False)
ds_rect   = xr.open_dataset(file_ex_ocn_rect, decode_times=False)
ds_had    = xr.open_dataset(file_HadISST    , decode_times=False)

In [ ]:
RMASK_ocn = RMASK_ocn.rename({'TLAT'    : 'lat', 'TLONG'    : 'lon'})
ds_rect   = ds_rect  .rename({'t_lat'   : 'lat', 't_lon'    : 'lon'})
ds_had    = ds_had   .rename({'latitude': 'lat', 'longitude': 'lon'})

In [ ]:
# replace NaNs in continents with values from original grid file
lats,lons = generate_lats_lons('ocn')
RMASK_ocn['lat'].values = lats
RMASK_ocn['lon'].values = lons

In [ ]:
%%time
# 40 sec
regridder_rect = xe.Regridder(RMASK_ocn, ds_rect, 'nearest_s2d', reuse_weights=True, periodic=True)
regridder_had  = xe.Regridder(RMASK_ocn, ds_had , 'nearest_s2d', reuse_weights=True, periodic=True)

### REGION_MASK

In [ ]:
da_rect = regridder_rect(RMASK_ocn).astype('int64').rename({'lat': 't_lat'   , 'lon': 't_lon'    })
da_had  = regridder_had (RMASK_ocn).astype('int64').rename({'lat': 'latitude', 'lon': 'longitude'})

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(12,4))
RMASK_ocn.where(RMASK_ocn>0).plot(vmin=0, ax=ax[0])
da_rect  .where(da_rect  >0).plot(vmin=0, ax=ax[1])
da_had   .where(da_had   >0).plot(vmin=0, ax=ax[2])
plt.tight_layout()

In [ ]:
# da_rect.to_netcdf(f'{file_RMASK_ocn_rect}')
# da_had .to_netcdf(f'{file_RMASK_ocn_had}' )

## using `FieldAnalysis` class

In [ ]:
from analysis import FieldAnalysis

In [ ]:
ds_low = xr.open_dataset(file_ex_ocn_lpd, decode_times=False)

In [ ]:
FA = FieldAnalysis()

In [ ]:
from xr_DataArrays import dll_coords_names

In [ ]:
dll_coords_names('ocn_low')

In [ ]:
ds_low.TEMP[0,0,:,:]

In [ ]:
ds = FA.regrid(ds_low.TEMP[0,0,:,:], ds_had.sst[0,:,:], method='bilinear')

In [ ]:
ds_had.sst

In [ ]:
((ds+ds_had.sst[0,:,:])/2).plot(vmin=-2, vmax=32)

In [ ]:
ds_low.TLONG.plot()

In [ ]:
ds.longitude.plot()

In [ ]:
plt.figure(figsize=(16,12))
ds.plot(vmin=-2, vmax=32)

In [ ]:
from xr_DataArrays  import xr_AREA
from regions import boolean_mask

In [ ]:
A = ds.copy()
B = ds_had.sst[0,:,:]
C = xr_AREA('ocn_had')
MASK = boolean_mask('ocn_had', 0)

In [ ]:
E = np.any(np.array([np.isnan(A).values, np.isnan(B).values, (MASK==0).values]), axis=0)

In [ ]:
A = xr.where(E, np.nan, A)
B = xr.where(E, np.nan, B)
C = xr.where(E, np.nan, C)

In [ ]:
A.plot()

In [ ]:
AA = A.stack(z=('latitude', 'longitude')).dropna(dim='z')
BB = B.stack(z=('latitude', 'longitude')).dropna(dim='z')
CC = C.stack(z=('latitude', 'longitude')).dropna(dim='z')

In [ ]:
from statsmodels.stats.weightstats import DescrStatsW

In [ ]:
d1 = DescrStatsW( np.array([AA.values, BB.values]).T, weights=CC)

In [ ]:
d1

In [ ]:
plt.scatter(AA, BB)

In [ ]:
d1.mean

In [ ]:
d1.corrcoef

In [ ]:
type(A)

In [ ]:
FA.spatial_correlation(A, ds_low.TEMP[0,0,:,:], selection={'latitude':slice(60,-60)})

In [ ]:
MASK

In [ ]:
MASK.sel({'latitude':slice(59.5,-59.5)})